# Method 2: using RT60s from AEC challenge dataset (ICASSP21)

In [1]:
import pandas as pd
import soundfile as sf
import IPython.display as ipd
import numpy as np
from os.path import join as pjoin
from os import listdir as ldr
import matplotlib.pyplot as plt
#import pyroomacoustics
import scipy.signal as sig
import scipy.stats as ss
import numpy as np
import tqdm
import pyroomacoustics as pra
from pyroomacoustics.parameters import constants



In [2]:
df = pd.read_csv('../AEC_challenge_ICASSP2021_RIRs.csv', encoding= 'unicode_escape')

In [3]:
room_names = list(set(list(df['alias'])))

In [4]:
bands = np.array([31.6228, 63.0975, 125.8925, 251.1886, 501.1872, 1000, 1995.2623, 3981.0717, 7943.2823, 15848.9319])

In [5]:
# we parse the dataframe 
real_rt60s = []

real_rt60Fs = []

errors = 0
for i in range(len(room_names)):
    room = df[df['alias'] == room_names[i]]
    channels = list(set(room['channel']))
    for channel in channels:
        try:
            rec = room[room['channel']==channel]
            b1 = rec[rec['frequency']=='31.6228']['RT60'].values[0]
            b2 = rec[rec['frequency']=='63.0957']['RT60'].values[0]
            b3 = rec[rec['frequency']=='125.8925']['RT60'].values[0]
            b4 = rec[rec['frequency']=='251.1886']['RT60'].values[0]
            b5 = rec[rec['frequency']=='501.1872']['RT60'].values[0]
            b6 =rec[rec['frequency']=='1000']['RT60'].values[0]
            b7 = rec[rec['frequency']=='1995.2623']['RT60'].values[0]
            b8 = rec[rec['frequency']=='3981.0717']['RT60'].values[0]
            b9 = rec[rec['frequency']=='7943.2823']['RT60'].values[0]
            if not rec[rec['frequency']=='15848.9319']['RT60'].empty:
                b10 = rec[rec['frequency']=='15848.9319']['RT60'].values[0]
            else:
                b10 = b9
            real_rt60s.append(rec[rec['frequency']=='wideband']['RT60'].values[0])
            #real_rt60Fs.append(np.array([b1, b2, b3, b4, b5, b6, b7, b8, b9, b10]))
            real_rt60Fs.append(np.array([b3, b4, b5, b6, b7, b8]))
        except:
            errors += 1
real_rt60s = np.array(real_rt60s)
real_rt60Fs = np.array(real_rt60Fs)

In [6]:
real_rt60s = np.array(real_rt60s)
real_rt60Fs = np.array(real_rt60Fs)

In [8]:
real_rt60Fs.shape

(4495, 6)

In [ ]:
real_rt60Fs

In [ ]:
plt.plot(real_rt60Fs[8,:])
# real rt60s are not monotonically decreasing so i would model as independent random variables

In [ ]:
len(real_rt60s)

In [ ]:
bands = bands[2:8]

In [ ]:
plt.figure(figsize=[8,12])
for i, band in enumerate(bands):
    plt.subplot(5,2, i+1)
    plt.hist(real_rt60Fs[:, i], 100);
    plt.grid(True)
    plt.xlabel('RT60['+str(band)+'Hz]')
    plt.ylabel('count')
    plt.tight_layout()

# Computing RIRs with RT60>4 is too computationally expensive. We'll filter al the IRs larger than that

In [ ]:
fltrd = []
for i in real_rt60Fs:
    if not (any(i > 2.)):
        fltrd.append(i)

In [9]:
real_rt60Fs = np.array(fltrd)

NameError: name 'fltrd' is not defined

In [ ]:
plt.figure(figsize=[8,12])
for i, band in enumerate(bands):
    plt.subplot(5,2, i+1)
    plt.hist(real_rt60Fs[:, i], 100);
    plt.grid(True)
    plt.xlabel('RT60['+str(band)+'Hz]')
    plt.ylabel('count')
    plt.tight_layout()

In [10]:
# We will model each exponential for each sub-band
alphas = []
locs = []
betas = []
sim_rt60Fs = []
abs_rt60Fs = []
for idx, band in enumerate(real_rt60Fs.T):
    fit_alpha, fit_loc, fit_beta = ss.gamma.fit(band)
    alphas.append(fit_alpha)
    betas.append(fit_beta)
    locs.append(fit_loc)
    sim_rt60Fs.append(ss.gamma.rvs(fit_alpha, fit_loc, fit_beta, size=100000))
    abs_rt60Fs.append(ss.gamma.rvs(fit_alpha, 0, fit_beta, size=100000))

In [11]:
ss.gamma.fit(band)

(0.48431545000447535, 0.0025245494070957994, 1.8648233543657953)

In [ ]:
sim_rt60Fs = np.array(sim_rt60Fs).T
abs_rt60Fs = np.array(abs_rt60Fs).T

In [ ]:
plt.figure(figsize=[8,12])
for i, band in enumerate(bands):
    plt.subplot(5,2, i+1)
    plt.hist(real_rt60Fs[:, i], 100, density=True, alpha=0.5);
    plt.hist(sim_rt60Fs[:, i], 100, density=True, alpha=0.5);
    #plt.hist(abs_rt60Fs[:, i], 100, density=True, alpha=0.5);
    plt.legend(['real', 'sim'])
    plt.grid(True)
    plt.xlabel('RT60['+str(band)+'Hz]')
    plt.ylabel('count')
    plt.tight_layout()
    plt.xlim([0,3])
plt.savefig('rt60Fs_sampling_distribution.pdf')

In [ ]:
alphas

In [ ]:
betas

In [ ]:
# now we adapt the function to generate a single rt60 vector online
def get_6band_rt60_vector():
    np.random.seed() #we randomize so multiprocessing doesn't yield same RT60s
    alphas = np.array([1.7196874268124676,
                         1.6152228672267106,
                         1.9318203836226113,
                         2.55718115999814,
                         4.176814897493042,
                         2.4892656080814346])
    betas = np.array([0.38685390302225775,
                         0.24453641709737417,
                         0.14321372785643122,
                         0.10453218827453133,
                         0.08678871224845529,
                         0.18290733668646034])
    sim_rt60Fs = []
    for i in range(len(alphas)):
        sim_rt60Fs.append(np.random.gamma(alphas[i], betas[i], 1))
    return np.array(sim_rt60Fs).squeeze()

In [ ]:
get_6band_rt60_vector()